# Prediction of Readmission of Diabetic Patients from 1999-2008

## Overview

This project will create classification models to predict hospital readmissions regarding patients with diabetes through data provided by Virginia Commonwealth University (downloaded from kaggle) from 1999-2008 with 130 U.S. hospitals. Predicting high risk patients will provide valuable information to the careproviders to better prepare future care needs of diabetic patients.

## Business Problem

Determining treatment effectiveness within diabetic pateints is an ardous process. Therefore, predicting readmission possibility of patients would allow care-providers to determine treatment qaulity, effectiveness, and prepare further treatment plans if necessary. Also, classifying high risk patients would determine ineffective or errorneous treatments, reducing treatment risks in the future. This prediction model will provide insight to treatment effecitiveness, cost reduction method, and reduced medical risks to careproviders. Hospitals and insurance angency can utilize this model to increase logistical and financial support for highly effective treatments and high risk patients.

## Data

The data provided by Virginia Commonwealth University collected from 130 U.S. hospitals over the years of 1999-2008 on diabetic pateints. The data is separated by unique patient ID and provides features such as number of medications, hospital stays, and sex. The readmission status was defined as "No", no readmission in that year, "<30", readmitted within 30 days of last visit, and ">30", readmitted after 30 days period.

A supporting documetion, description.pdf, Impatct of HbA1c Measurement on Hospital Readmission Rates: Analysis of 70,000 Clinical Databse Patient Records was also provided as reference.

## Methods

1. Clean data set by creating dummy variables for categorical data 
    - Diagnosis categorized per ICD9 code group defined by related study refere to decription.pdf page 5 table 2.
    - Dropped irrelevant or largely missing data columns such as pateint ID and weight.
2. Perform EDA on cleaned data to gain understanding of statistical significance for feature engineering. Also create data visualizatoin of statistically signifiant features.
3. Create baseline models for iterative improvements using recall as the scoring metric.
4. Select best perfomring model and crate final prediction for model performance analysis.
5. Compare feature weights and ranks among the best performing model to determine significant indicators of readmission prediction. 

## Library Import

In [2]:
import pickle
import pandas as pd
import json
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Exploratory Data Analysis

### T-test on Select Features

alpha = 0.5 <br>

Selected Features = Number of Medication, Procedure, and Lab Procedures <Br>

Null Hypothesis: Selected Features are not significant factors determining readmission. <br>
Alternative Hypothesis: Selected Features are significant factors determining readmission. <br>

In [ ]:
features = ["num_medications", "num_procedures", "num_lab_procedures", "number_outpatient", "number_inpatient"]
alpha = 0.05

for feature in features:
    #slice data
    readmit_0 = clean_df[clean_df.readmitted == 0][feature]
    readmit_1 = clean_df[clean_df.readmitted == 1][feature]
    readmit_2 = clean_df[clean_df.readmitted == 2][feature]
    
    #series of t-test
    num_med_0_1 = stat.ttest_ind(readmit_0, readmit_1, equal_var = False)[1]
    num_med_1_2 = stat.ttest_ind(readmit_1, readmit_2, equal_var = False)[1]
    num_med_0_2 = stat.ttest_ind(readmit_0, readmit_2, equal_var = False)[1]
    
    if num_med_0_1 <= alpha:
        print(f"{feature} T-test for readmission 0 vs 1 P-value: ", num_med_0_1)
        print("Reject Null Hypothesis", "\n")
    else:
        print(f"{feature} T-test for readmission 0 vs 1 P-value: ", num_med_0_1)
        print("FAILURE to Reject Null Hypothesis", "\n")
    if num_med_1_2 <= alpha:
        print(f"{feature} T-test for readmission 1 vs 2 P-value: ", num_med_1_2)
        print("Reject Null Hypothesis", "\n")
    else:
        print(f"{feature} T-test for readmission 1 vs 2 P-value: ", num_med_1_2)
        print("FAILURE to Reject Null Hypothesis", "\n")
    if num_med_0_2 <= alpha:
        print(f"{feature} T-test for readmission 0 vs 2 P-value: ", num_med_0_2)
        print("Reject Null Hypothesis", "\n")
    else:
        print(f"{feature} T-test for readmission 0 vs 2 P-value: ", num_med_0_2)
        print("FAILURE to Reject Null Hypothesis", "\n")

#### T-test Results

- Null hypothesis can be rejected for number of medications, statistical significance were found between number of medication in determining readmission status

- Failed to reject null hypothesis on statistical significance of number of procedures determining readmission within or after 30 days. 

- Failed to reject null hypothesis on statistical significance of number of lab procedures determining readmission within or after 30 days. 

- Number of inpatient T-test displaying p-values of 0 demonstrated very large frequency imbalance between the three groups. 

### Violin Plot of significant features

Number of medication, number of inpatient visits, and number of outpatient visits were all significant factors in determining readmission possibility for all outcomes. 

#### Number of Medications

In [ ]:
#set color?
sns.set_palette("rocket")
#data slice
x0 = clean_df[clean_df.readmitted == 0]["num_medications"]
x1 = clean_df[clean_df.readmitted == 1]["num_medications"]
x2 = clean_df[clean_df.readmitted == 2]["num_medications"]

#set subplots
fig, ax = plt.subplots(3,1, figsize=(6,8))
#subplot 1
sns.violinplot(x0, color="lightgrey", ax=ax[0], inner="quartile")
ax[0].set_title("No Readmission")
ax[0].set_xlim(-5,85)
ax[0].set_xlabel("# of Medications")

#subplot 2
sns.violinplot(x1, color="cornflowerblue", ax=ax[1], inner="quartile")
ax[1].set_title("Readmission in <30 Days")
ax[1].set_xlim(-5,85)
ax[1].set_xlabel("# of Medications")

#subplot 3
sns.violinplot(x2, color="blue", ax=ax[2], inner="quartile")
ax[2].set_title("Readmission in >30 Days")
ax[2].set_xlim(-5,85)
ax[2].set_xlabel("# of Medications")

#prettify
plt.suptitle("Distribution of Medication Data per Readmission Status")
plt.tight_layout(); #make it neat

plt.show();

Number of medications have small data spared difference when divided by readmission status. First, the readmission within <30 days had highest spread in number of medication. Second, the mean medication among this group was higher than mean of other readmission status. This can be interpreted as high risk (more likely to return) patients are assigned more medicaiton to support them. 

In [ ]:
# data slice
x0 = clean_df[clean_df.readmitted == 0]["number_inpatient"]
x1 = clean_df[clean_df.readmitted == 1]["number_inpatient"]
x2 = clean_df[clean_df.readmitted == 2]["number_inpatient"]

# plot
fig, ax = plt.subplots(3,1, figsize=(6,8))
# subplot 1
sns.violinplot(x0, color="lightgrey", alpha=0.7, ax=ax[0])
ax[0].set_title("No Readmission")
ax[0].set_xlim(-2, 25)
ax[0].set_xlabel("# of Inpatient")

# subplot 2
sns.violinplot(x1, color="cornflowerblue", alpha=0.7, ax=ax[1])
ax[1].set_title("Readmission in <30 Days")
ax[1].set_xlim(-2, 25)
ax[1].set_xlabel("# of Inpatient")

# subplot 3
sns.violinplot(x2, color="blue", alpha=0.7, ax=ax[2])
ax[2].set_title("Readmission in >30 Days")
ax[2].set_xlim(-2, 25)
ax[2].set_xlabel("# of Inpatient")

# prettify
plt.suptitle("Distribution of Inpatient Days per Readmission Status")
plt.tight_layout(); #make it neat

plt.show();

Number of inpatient days demonstrated significant data spread when separated by readmission status. Readmission within 30 days shows higher frequency and spared of number of inpatient days. Most patients with no readmission had 0 inpatient visits. 

### Stacked Bar Plots

#### Age

In [ ]:
# Age summary
print('Age Summary')

print('\n')
print(clean_df.groupby(['age','readmitted']).age.count().unstack())

# Age class visualization
ageclass = clean_df.groupby(['age','readmitted']).age.count().unstack()
p1 = ageclass.plot(kind = 'bar', stacked = True, 
                   title = 'Patients by Age: Not Readmitted vs. Readmitted', 
                   color = ['grey','lightgreen', 'blue'], alpha = .70)
p1.set_xlabel('Age')
p1.set_ylabel('Readmitted')
p1.legend(['No','<30', '>30'])
#plt.savefig('../image/Age_Readmiited.png')
plt.show()

The age category of patients were skewed towards the older (>50) patients.

#### Probability of Readmittance by Age

In [ ]:
ppa = clean_df.groupby(['age','readmitted']).age.count().unstack()
spa = clean_df.groupby(['age','readmitted']).age.count().unstack()
tpa = clean_df.groupby(['readmitted', 'age']).readmitted.count().unstack().sum()
for x in range(0, 3):
    ppa[x] = spa[x] / tpa

print('Probability of Readmittance by Age Summary')
print('\n')
print(ppa)

# Probability of Readmittance by Age visualization
p1 = ppa.plot(kind = 'bar', stacked = True, 
                   title = 'Patients by Age: Not Readmitted vs. Readmitted', 
                   color = ['grey','lightgreen', 'blue'], alpha = .70)
p1.set_xlabel('Age')
p1.set_ylabel('Readmitted')
p1.legend(['No','<30', '>30'])
#plt.savefig('../image/Percentage_Age_Readmitted.png')
plt.show()

From the graph above, we can see that for the most part, as age increases, the probability of readmittance increases. The probability of readmittance after 30 days greater then before 30 days.

#### Length of Hospital Stay

In [ ]:
# Hospital Stay class summary
print('Hospital Stay Summary')

print('\n')
print(clean_df.groupby(['time_in_hospital','readmitted']).time_in_hospital.count().unstack())

# Hospital Stay visualization
hospital_stay = clean_df.groupby(['time_in_hospital','readmitted']).time_in_hospital.count().unstack()
p1 = hospital_stay.plot(kind = 'bar', stacked = True, 
                   title = 'Patients by Hospital Stay: Not Readmitted vs. Readmitted', 
                   color = ['grey','lightgreen', 'blue'], alpha = .70)
p1.set_xlabel('Hospital Stay')
p1.set_ylabel('Readmitted')
p1.legend(['No','<30', '>30'])
#plt.savefig('../image/Hospital_Stay_Readmitted.png')
plt.show()

There is a skew in the frequency of hospital stay lengths towards stays less than 5 days.

#### Probability of Readmittance by Length of Hospital Stay

In [ ]:
pph = clean_df.groupby(['time_in_hospital','readmitted']).time_in_hospital.count().unstack()
sph = clean_df.groupby(['time_in_hospital','readmitted']).time_in_hospital.count().unstack()
tph = clean_df.groupby(['readmitted', 'time_in_hospital']).readmitted.count().unstack().sum()
for x in range(0, 3):
    pph[x] = sph[x] / tph

print('Probability of Readmittance by Length of Hospital Stay Summary')
print('\n')
print(pph)

# Probability of Readmittance by Length of Hospital Stay Visualization
p1 = pph.plot(kind = 'bar', stacked = True, 
                   title = 'Patients by Hospital Stay: Not Readmitted vs. Readmitted', 
                   color = ['grey','lightgreen', 'blue'], alpha = .70)
p1.set_xlabel('Hospital Stay')
p1.set_ylabel('Readmitted')
p1.legend(['No','<30', '>30'])
#plt.savefig('../image/Percentage_Hospital_Stay_Readmitted.png')
plt.show()

As the hospital stay increases, the probability of readmittance increses as well.

#### Number of Diagnoses

In [ ]:
# diagnosis summary
print('Number of Diagnoses Summary')

print('\n')
print(clean_df.groupby(['number_diagnoses','readmitted']).number_diagnoses.count().unstack())

# gender visualization
nodclass = clean_df.groupby(['number_diagnoses','readmitted']).number_diagnoses.count().unstack()
p1 = nodclass.iloc[0:9, :].plot(kind = 'bar', stacked = True, 
                   title = 'Patients by Number of Diagnoses: Not Readmitted vs. Readmitted', 
                   color = ['grey','lightgreen', 'blue'], alpha = .70)
p1.set_xlabel('Number of Diagnoses')
p1.set_ylabel('Readmitted')
p1.legend(['No','<30', '>30'])
#plt.savefig('../image/NoD_Readmitted.png')
plt.show()

Due to the lack of data for patients with diagnoses greater than 9, the graph only shows the first 9 bars. Patients with 9 diagnoses are most frequent in the dataset. 

#### Probability of Readmittance by Number of Diagnoses

In [ ]:
ppnd = clean_df.groupby(['number_diagnoses','readmitted']).number_diagnoses.count().unstack()
spnd = clean_df.groupby(['number_diagnoses','readmitted']).number_diagnoses.count().unstack()
tpnd = clean_df.groupby(['readmitted', 'number_diagnoses']).readmitted.count().unstack().sum()
for x in range(0, 3):
    ppnd[x] = spnd[x] / tpnd

print('Probability of Readmittance by Number of Diagnoses Summary')
print('\n')
print(ppnd)

# Probability of Readmittance by Number of Diagnoses Visualization
# ignoring values greater than 9, due to insignificant amount of data
p1 = ppnd.iloc[0:9, :].plot(kind = 'line', stacked = True, 
                   title = 'Patients by Number of Diagnoses: Not Readmitted vs. Readmitted', 
                   color = ['grey','lightgreen', 'blue'], alpha = .70)
p1.set_xlabel('Number of Diagnoses')
p1.set_ylabel('Readmitted')
plt.fill_between(ppnd.iloc[0:9, :].index, ppnd.iloc[0:9, :][0], color = 'gray', alpha = 0.2) # fill in between the lines with color of readmittance
plt.fill_between(ppnd.iloc[0:9, :].index, ppnd.iloc[0:9, :][0] + ppnd.iloc[0:9, :][1], ppnd.iloc[0:9, :][0], color = 'lightgreen', alpha = 0.2)
plt.fill_between(ppnd.iloc[0:9, :].index, 1 - ppnd.iloc[0:9, :][2], 1, color = 'blue', alpha = 0.2)
p1.set_ylim(0.4, 1) # Taking a closer look at change in percentage by only looking above 40%
p1.legend(['No','<30', '>30'])
#plt.savefig('../image/Percentage_NoD_Readmitted.png')
plt.show()

The probability of being readmitted generally increases as the number of diagnoses increases.

## Models

In [11]:
# run below to unzip random forest model
# !unzip model\best_grid_rf_clf.zip

### Decision tree model with all features

In [7]:
with open(r"model\best_tree.pickle", "rb") as best_tree:
    grid_tree_all_clf = pickle.load(best_tree)

### Decision tree model with select features and feature names

In [8]:
with open(r"model\best_feature_tree_fit.pickle", "rb") as best_tree_select:
    grid_tree_select_clf = pickle.load(best_tree_select)

In [13]:
# load json file with all features 
with open(r"model\dt_features.json") as feature_json:
    feature_load = json.load(feature_json)

#all features used for gridsearch from json file
features = feature_load["features"]

### Random forest model

In [14]:
with open(r"model\best_grid_rf_clf.pickle", "rb") as rf_clf:
    grid_rf_clf = pickle.load(rf_clf)